In [75]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:
import pandas as pd
import os
import sys
sys.path.append('../src/')
import vtn
from bokeh.io import output_notebook
import itertools 
output_notebook()

from ipywidgets import interact, interactive_output, interact_manual
import numpy as np
from scipy import optimize

Loading BokehJS ...

In [102]:
def load_data(n, datadir, filename):
    d_dict = {}
    for i in range(1,n+1):
        d_dict[f'{i}'] = pd.read_csv(os.path.join(datadir, filename.format(i=i)))
    return d_dict

# Initilize `VariableTimeNormalization` object

In [103]:
# Pair of experiments
exp_pairs = {
    'A':{
        'exps':['1','2'], 
        'type':'variable', 
        'colors':['blue', 'red']
    },
    'B':{
        'exps':['1','3'], 
        'type':'variable', 
        'colors':['blue', 'green']
    },
    'cat':{
        'exps':['1','4'], 
        'type':'constant', 
        'colors':['blue', 'orange']
    }
}

# Reaction orders with respect to each component
rxn_orders = {'A':1, 'B':1, 'cat':1}

# Initialize vtn object
exp = vtn.VariableTimeNormalization(
    df0=load_data(4, '../data/', 'exp_{i}.csv'), 
    exp_pairs=exp_pairs, 
    rxn_orders=rxn_orders, 
    product_name='P',
    
)

# Determine reaction orders
We can plot the the total variation in the absolute value of the discrete difference in the data over a range of reaction orders. The minimum of this plot indicates the order of the reaction.

There are two plotting functions available for total variation plotting:
- `plot_tv()`: plot total variable function `tv()` over specified range
- `plot_min_tv()`: find global minimum of `tv()` by brute force over specified grid

In [65]:
vtn.plot_tv(
    exp=exp,
    comps=['A','B','cat'],
    x0_arange=[0,2,0.25]
)

In [105]:
vtn.plot_min_tv(
    exp=exp,
    comps=['A','B','cat'],
    x0_arange=(0,2),
    n=25
)

# Generate VTN plots

All the data required to generate VTN plots are stored in `.df_vtn` dictionary. They keys of the dictionary correspond to experiment number. i.e. `.df_vtn['1']` is the DataFrame containing data for experiment `1`.

There are two plotting methods available to `VariableTimeNormalization` object.
- `plot_vtn()`: used for pairwise experiment plotting for determining order with respect to a component
- `plot_kobs()`: used for plotting product against normalized time from all exeriments to determine $k_{obs}$

In [7]:
exp = vtn.VariableTimeNormalization(
    load_data(4, '../data/', 'exp_{i}.csv'), 
    exp_pairs=exp_pairs, 
    rxn_orders={'A':1, 'B':1, 'cat':1}, 
    product_name='P'
)

exp.plot_vtn('A')
exp.plot_kobs()
exp.df_vtn['3']

,t,A,B,P,cat,t-1,[cat]^1,t[cat]^1,A-1,[A]^1,[A]^1∆t,∑[A]^1∆t,B-1,[B]^1,[B]^1∆t,∑[B]^1∆t,∆t,∑[A]^1[B]^1[cat]^1∆t
0,0.00,0.999,0.801,0.000,0.01,NaN,0.01,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.33,0.915,0.715,0.086,0.01,0.00,0.01,0.0033,0.999,0.9570,0.315810,0.315810,0.801,0.7580,0.250140,0.250140,0.33,0.002394
2,1.00,0.786,0.585,0.213,0.01,0.33,0.01,0.0100,0.915,0.8505,0.569835,0.885645,0.715,0.6500,0.435500,0.685640,0.67,0.006098
3,1.92,0.663,0.464,0.337,0.01,1.00,0.01,0.0192,0.786,0.7245,0.666540,1.552185,0.585,0.5245,0.482540,1.168180,0.92,0.009594
4,3.08,0.560,0.359,0.441,0.01,1.92,0.01,0.0308,0.663,0.6115,0.709340,2.261525,0.464,0.4115,0.477340,1.645520,1.16,0.012513
5,4.38,0.482,0.284,0.515,0.01,3.08,0.01,0.0438,0.560,0.5210,0.677300,2.938825,0.359,0.3215,0.417950,2.063470,1.30,0.014690
6,6.29,0.410,0.211,0.591,0.01,4.38,0.01,0.0629,0.482,0.4460,0.851860,3.790685,0.284,0.2475,0.472725,2.536195,1.91,0.016799
7,11.54,0.310,0.108,0.691,0.01,6.29,0.01,0.1154,0.410,0.3600,1.890000,5.680685,0.211,0.1595,0.837375,3.373570,5.25,0.019813
8,22.92,0.237,0.038,0.763,0.01,11.54,0.01,0.2292,0.310,0.2735,3.112430,8.793115,0.108,0.0730,0.830740,4.204310,11.38,0.022085
9,41.67,0.208,0.009,0.792,0.01,22.92,0.01,0.4167,0.237,0.2225,4.171875,12.964990,0.038,0.0235,0.440625,4.644935,18.75,0.023066


# OLS to determine $k_{obs}$

- `.kobs_line_format` can be specified when initializing the `vtn` object. This is the formula used for regression analysis to determine $k_{obs}$. This follows `R`-programming stype notation. By default this is set to `y ~ x-1`. This corresponds to fitting $y = mx$. The `-1` forces intercept to `0`.

- `.df_kobs_reg` is the DataFrame that contains data used for regression analysis to determine $k_{obs}$

- `.kobs_line` is a `statsmodel` object containing the OLS line. The output of OLS analysis can be generated using the `.summary()` method.

In [8]:
exp.kobs_line_format

'y ~ x-1'

In [9]:
exp.df_kobs_reg.head()

,P,∑[A]^1[B]^1[cat]^1∆t,y,x
1,0.150,0.004313,0.150,0.004313
2,0.273,0.007794,0.273,0.007794
3,0.388,0.011115,0.388,0.011115
4,0.477,0.013669,0.477,0.013669
5,0.534,0.015263,0.534,0.015263


In [10]:
exp.kobs_line.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 8.327e+05
Date:                Thu, 14 Feb 2019   Prob (F-statistic):           3.46e-78
Time:                        20:16:07   Log-Likelihood:                 156.22
No. Observations:                  36   AIC:                            -310.4
Df Residuals:                      35   BIC:                            -308.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x             34.8277      0.038    912.498      0.000      34.750      34.905
==============================================================================
Omnibus:                       22.711   Durbin-Watson:                   1.270
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.440
Skew:                          -1.573   Prob(JB):                     1.65e-09
Kurtosis:                       7.130   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""